In [2]:
library(raster)
library(caret)
library(randomForest)

NameError: name 'library' is not defined

In [ ]:
set.seed(1000)
RD.files <- './Data/Reference_Data.shp'
RD <- shapefile(RD.files)

data.folder <- './Data/Predictors'

file.names <- list.files(data.folder, pattern='f_') 
file.paths <- paste(data.folder, file.names, sep="/")


predictors <- stack(file.paths)
crs(predictors) <- crs(RD)

In [ ]:

pred.val <- extract(predictors,RD, method = 'bilinear')
dataset <- as.data.frame(pred.val)
dataset$CLASS <- RD$Class

In [ ]:

nc <- length(unique(dataset$CLASS)) #number of target classes
c1 <- length(colnames(dataset)) #number of predictors
sample <- rep(ceiling(0.75*min(table(dataset$CLASS))),nc)

In [ ]:

subsets <- seq(2,ncol(dataset)-1,3)
set.seed(1000)
ctrl <- rfeControl(functions = rfFuncs,
               method = "repeatedcv",
               number = 10,
               repeats = 5,
               verbose = F)

prProfile <- rfe(x=dataset[, c(1:ncol(dataset)-1)], y=as.factor(dataset$CLASS),
                 sizes = subsets,
                 rfeControl = ctrl)

pr <- predictors(prProfile)[1: prProfile$bestSubset]

In [ ]:

set.seed(1000)
ctrl <- trainControl(method = "repeatedcv",number = 10, repeats = 5)
dataset <- dataset[,c(pr,'CLASS')]

mtry <- 1:sqrt(ncol(dataset))
tunegrid <- expand.grid(.mtry=mtry)
rfDownsampled <- train(as.factor(CLASS) ~ ., data = dataset,
                         method = "rf",
                         ntree = 1500,
                         tuneGrid = tunegrid,
                         metric = "Accuracy",
                         strata = as.factor(dataset$CLASS),
                         trControl = ctrl,
                         sampsize = sample)

In [ ]:

map <- predict(disaggregate(predictors, c(2,2), method = 'bilinear'),rfDownsampled)

plot(map)
writeRaster(map, './Data/Results/VegetationMap.tif',datatype = 

In [ ]:

writeRaster(map, './Data/Results/VegetationMap.tif',datatype = 'INT2S', overwrite=TRUE)

write.table(rfDownsampled$results,dec =',','./Data/Results/TOPORes.txt')
write.table(rfDownsampled$finalModel$confusion,dec =',', './Data/Result